# Preprocesamiento de Imágenes - Modelo CNN Melanoma

Este notebook muestra cómo usar el módulo de preprocesamiento para preparar las imágenes.

In [ ]:
import os
import sys
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt

# Montar Google Drive
drive.mount('/content/drive')

In [ ]:
# Agregar src al path
sys.path.append('/content/drive/MyDrive/TIC_CNN_Modelo_Melanoma')

from src.data import create_data_generators, create_data_flow_from_dataframe
from src.config.config import CSV_SPLIT_FOLDER, BATCH_SIZE, IMAGE_SIZE

## 1. Cargar los datasets divididos

In [ ]:
# Cargar los CSVs generados anteriormente
train_df = pd.read_csv(os.path.join(CSV_SPLIT_FOLDER, "train.csv"))
val_df = pd.read_csv(os.path.join(CSV_SPLIT_FOLDER, "val.csv"))
test_df = pd.read_csv(os.path.join(CSV_SPLIT_FOLDER, "test.csv"))

print(f"Train: {len(train_df)} imágenes")
print(f"Val: {len(val_df)} imágenes")
print(f"Test: {len(test_df)} imágenes")

## 2. Crear generadores de datos

Los generadores aplican:
- **Train**: Augmentation (rotación, zoom, flips) + Normalización ImageNet
- **Val/Test**: Solo normalización ImageNet

In [ ]:
train_datagen, val_test_datagen = create_data_generators()

print("Generadores creados:")
print(f"  - Train: con augmentation")
print(f"  - Val/Test: solo normalización")

## 3. Crear flujos de datos

In [ ]:
train_generator = create_data_flow_from_dataframe(
    train_datagen, 
    train_df, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

val_generator = create_data_flow_from_dataframe(
    val_test_datagen, 
    val_df, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

test_generator = create_data_flow_from_dataframe(
    val_test_datagen, 
    test_df, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

print(f"\nFlujos creados con batch_size={BATCH_SIZE}")
print(f"Train steps per epoch: {len(train_generator)}")
print(f"Val steps per epoch: {len(val_generator)}")
print(f"Test steps: {len(test_generator)}")

## 4. Visualizar ejemplos de augmentation

In [ ]:
# Obtener un batch de entrenamiento
sample_batch_x, sample_batch_y = next(train_generator)

# Visualizar 8 imágenes con augmentation
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.ravel()

for i in range(8):
    # Denormalizar para visualización (aproximado)
    img = sample_batch_x[i]
    img = (img - img.min()) / (img.max() - img.min())
    
    axes[i].imshow(img)
    axes[i].set_title(f"Label: {int(sample_batch_y[i])}")
    axes[i].axis('off')

plt.suptitle('Ejemplos de imágenes con augmentation (Train)', fontsize=16)
plt.tight_layout()
plt.show()

## 5. Verificar forma de los datos

In [ ]:
print(f"Forma del batch de imágenes: {sample_batch_x.shape}")
print(f"Forma del batch de etiquetas: {sample_batch_y.shape}")
print(f"Target size configurado: {IMAGE_SIZE}")
print(f"\nRango de valores después de preprocesamiento:")
print(f"  Min: {sample_batch_x.min():.2f}")
print(f"  Max: {sample_batch_x.max():.2f}")
print(f"  Mean: {sample_batch_x.mean():.2f}")

## 6. Ejemplo de preprocesamiento individual (para inferencia)

In [ ]:
from src.data import load_and_preprocess_image

# Tomar una imagen de ejemplo
example_path = train_df['filepath'].iloc[0]
print(f"Procesando: {example_path}")

# Preprocesar para inferencia
processed_img = load_and_preprocess_image(example_path)

print(f"\nForma procesada: {processed_img.shape}")
print(f"Lista para modelo: {processed_img.shape == (1, *IMAGE_SIZE, 3)}")

## 7. (Opcional) Limpieza de artefactos

Ejemplo de eliminación de pelos usando morfología.

In [ ]:
from src.data import preprocess_with_cleaning
import cv2

# Cargar imagen original
example_img = cv2.imread(example_path)
example_img = cv2.cvtColor(example_img, cv2.COLOR_BGR2RGB)

# Procesar con limpieza
cleaned_img = preprocess_with_cleaning(example_path)

# Visualizar comparación
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(cv2.resize(example_img, IMAGE_SIZE))
axes[0].set_title('Original')
axes[0].axis('off')

# Denormalizar para visualización
cleaned_display = cleaned_img[0]
cleaned_display = (cleaned_display - cleaned_display.min()) / (cleaned_display.max() - cleaned_display.min())
axes[1].imshow(cleaned_display)
axes[1].set_title('Con limpieza de artefactos')
axes[1].axis('off')

plt.tight_layout()
plt.show()